# Lab 2
Use *genetic algorithms* (GA) to solve the set cover problem.

In [3]:
# given function to yield list of lists
import random

def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [4]:
# logging
import logging
#logging.basicConfig(format="%(message)s", level=logging.DEBUG)
logging.basicConfig(filename='logging.log', encoding='utf-8', level=logging.INFO)

## Solution outline, $(\mu + \lambda)$ algorithm
### (also $(\mu , \lambda)$ possible if `update_population_comma` is used instead of `update_population_plus`
1. Create initial population by selecting a random subset of the list of lists
2. Compute fitness to rank population
3. Choose parents using roulette wheel with ranking system
4. Generate offspring by cross-over, mutate offspring with low probability
5. Choose the best solutions of population and offspring (*or just best from offspring*)
6. Repeat step 2-5

In [5]:
# imports
from itertools import groupby
import random
import time

In [6]:
# helping functions

def lists_to_set(genome):
    """
    convert genome to set
    :param genome: the sub-lists with random integers between 0 and N-1
    :return: set of contained elements in the genome
    """
    list_elems = [single_elem for l in genome for single_elem in l]
    s = set(list_elems)
    return s

# find out how many duplicates there are in the population
def count_duplicates(genome):
    """
    Count how many duplicates there are in the genome
    :param genome: the sub-lists with random integers between 0 and N-1
    :return: the count
    """
    list_elems = [single_elem for l in genome for single_elem in l]
    duplicates = sum([len(list(group))-1 for key, group in groupby(sorted(list_elems))])
    return duplicates


In [55]:
# to initialize the population
def create_population(STATE_SPACE, GOAL):
    """
    Initialize the population.
    :param STATE_SPACE: List of lists generated from problem-function
    :param GOAL: set of integers from 0 to N-1
    :return: a list of tuples: (genome,fitness), for each individual in the population.
    """
    population = []
    for _ in range(POPULATION_SIZE):
        individual = []
        for _ in range(random.randint(1,len(STATE_SPACE))):
            l = random.choice(STATE_SPACE)
            if l not in individual: #check duplicates here
                individual.append(l)
        #individual = random.choices(STATE_SPACE,k=random.randint(1,len(STATE_SPACE)))
        fitness = compute_fitness(individual, GOAL)
        population.append((individual,fitness))
    return population

def compute_fitness(genome, GOAL):
    """
    fitness is a tuple of (-#of_elems_missing,-#duplicates) which should be maximized
    :param genome: the sub-lists with random integers between 0 and N-1
    :param GOAL: set of integers from 0 to N-1
    :return: the fitness
    """
    # violated constraints, i.e. how many elements are missing
    vc = GOAL.difference(lists_to_set(genome))
    duplicates = count_duplicates(genome)
    # it is worse to lack elements than having duplicates
    fitness = (-len(vc), -duplicates)
    return fitness

def goal_check(genome, GOAL):
    """
    Check if all required elements are in the genome
    :param genome: the sub-lists with random integers between 0 and N-1
    :param GOAL: set of integers from 0 to N-1
    :return: boolean value if goal reached or not
    """
    return GOAL==lists_to_set(genome)

def parent_selection(population):
    """
    parent selection using ranking system
    P(choose fittest parent) = POPULATION_SIZE/n_slots
    P(choose second fittest parent) = (POPULATION_SIZE-1)/n_slots
    ...
    P(choose least fit parent) = 1/n_slots
    :param population: list of individuals
    :return: parent to generate offspring
    """
    ranked_population = sorted(population, key=lambda t : t[1], reverse=True)
    # number of slots in spinning wheel = POPULATION_SIZE(POPULATION_SIZE+1)/2 (arithmetic sum)
    n_slots = POPULATION_SIZE*(POPULATION_SIZE+1)/2
    wheel_number = random.randint(1,n_slots)
    curr_parent = 0
    parent_number = POPULATION_SIZE
    increment = POPULATION_SIZE-1
    while wheel_number > parent_number:
        curr_parent +=1
        parent_number +=increment
        increment -= 1
    return ranked_population[curr_parent]

# make one child from each cross-over, and mutate with low prob
def cross_over(parent1, parent2, STATE_SPACE, mutation_prob = 0.1):
    """
    Compute cross-over between two selected parents. Mutate child with mutation_prob.
    :param parent1: individual
    :param parent2: individual
    :param STATE_SPACE: List of lists generated from problem-function
    :param mutation_prob: the probability to perform mutation
    :return: the child created
    """
    cut1 = random.randint(0,len(parent1[0]))
    cut2 = random.randint(0,len(parent2[0]))
    child = parent1[0][:cut1]+parent2[0][cut2:]
    if random.random() < mutation_prob:
        mutate(child, STATE_SPACE)
    return child


def mutate(child, STATE_SPACE):
    """
    Replace one list in the child with a random one from the state space.
    :param child:
    :param STATE_SPACE:
    :return: the mutated child
    """
    idx = random.randint(0,len(child))
    #child = child[:idx] + child[idx+1:] + STATE_SPACE[random.randint(0,len(STATE_SPACE)-1)]
    i = 0
    while i<10:
        i+=1
        if STATE_SPACE[random.randint(0,len(STATE_SPACE)-1)] not in child:
             child = child[:idx] + child[idx+1:] + STATE_SPACE[random.randint(0,len(STATE_SPACE)-1)]
             break
    return child

def update_population_plus(population, offspring):
    """
    Using the plus strategy to update population to next generation.
    :param population:
    :param offspring:
    :return: the best individuals in union(population, offspring)
    """
    tot = population + offspring
    ranked_population = sorted(tot, key=lambda t : t[1], reverse=True)
    return ranked_population[:POPULATION_SIZE]

def update_population_comma(offspring):
    """
    Using the plus strategy to update population to next generation.
    :param offspring:
    :return: the best individuals in from offspring
    """
    ranked_pop = sorted(offspring, key=lambda t : t[1], reverse=True)
    return ranked_pop[:POPULATION_SIZE]

def update_mutation_prob(best_solution, best_this_iter, mutation_param, it):
    """
    Update the mutation probability according to how the performance evolves. If no improvement, mutation probability increases (favour exploration). If improvement, mutation probability decreases (favour exploitation).
    :param best_solution: The best solution so far
    :param best_this_iter: The best solution of this generation
    :param mutation_param:
    :param it: iteration number
    :return: the new mutation probability
    """
    if best_solution[1] >= best_this_iter[1]:
        mutation_param +=1
    elif best_solution[1] >= best_this_iter[1] and mutation_param>0:
        mutation_param -= 1
    return mutation_param/(1+it), mutation_param

In [56]:
def solve_problem(N):
    STATE_SPACE = problem(N,seed=42)
    GOAL = set(range(N))
    population = create_population(STATE_SPACE, GOAL)
    best_sol = population[0] #to be updated after each iter
    found_in_iter = 0 #to be updated
    mutation_param = 1 #increase if solution doesn't improve
    mutation_prob = 0.1 #init value
    for i in range(ITERS):
        offspring = []
        for __ in range(OFFSPRING_SIZE):
            parent1, parent2 = parent_selection(population), parent_selection(population)
            child = cross_over(parent1,parent2, STATE_SPACE, mutation_prob)
            child_fitness = compute_fitness(child, GOAL)
            offspring.append((child,child_fitness))
        #population = update_population_plus(population, offspring)
        population = update_population_comma(offspring)
        best_curr = sorted(population, key=lambda l:l[1], reverse=True)[0]
        mutation_prob, mutation_param = update_mutation_prob(best_sol, best_curr, mutation_param, i)
        if goal_check(best_curr[0],GOAL) and best_curr[1] > best_sol[1]:
            best_sol = best_curr
            found_in_iter = i
    logging.info(f'Best solution found in {found_in_iter} iters and has weight {-best_sol[1][1]}')
    return best_sol



In [57]:
# main

# settings
POPULATION_SIZE = 50
OFFSPRING_SIZE = 300
ITERS = 100

for N in [5,10,20,100,1000]:
    best_sol = solve_problem(N)
    print(f'N = {N}')
    logging.info(f'The best weight for N = {N}: {-best_sol[1][1]+N}')


N = 5
N = 10
N = 20
N = 100
N = 1000
